In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

Loaded environment variables from .env file.


In [2]:
import torch
from research_tools import get_gpus_available
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer


os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])
model_dtype = torch.bfloat16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device.type == "cuda", "No GPU available."

model_name = "meta-llama/Meta-Llama-3-8B"

model: LlamaForCausalLM = AutoModelForCausalLM.from_pretrained(
    model_name, token=hf_access_token, torch_dtype=model_dtype
)
model = model.to(device)

tokenizer: LlamaTokenizer = AutoTokenizer.from_pretrained(
    model_name, token=hf_access_token
)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
# from peft import get_peft_model, LoraConfig


# lora_rank = 64
# lora_alpha = 8

# lora_config = LoraConfig(
#     r=lora_rank,
#     lora_alpha=lora_alpha,
#     target_modules=["q_proj", "v_proj"],
# )

# model = get_peft_model(model, lora_config)

In [4]:
from unlearn_order.dataset import load_dataset

data_dir = Path("../data/random_bd")

splits = list(range(10))
n_train = 1
n_val = 1

train_files = [f"split_{splits[i]}.jsonl" for i in range(n_train)]
val_files = [f"split_{splits[i]}.jsonl" for i in range(n_train, n_train + n_val)]
combined_files = train_files + val_files

train_dataset = load_dataset(data_dir, train_files)
val_dataset = load_dataset(data_dir, val_files)
combined_dataset = load_dataset(data_dir, combined_files)

In [5]:
from unlearn_order.common import ExpConfig, Task, TaskType, DatasetType

cfg = ExpConfig(
    lr=3e-6,
    data_dir="../data/random_bd",
    task_order=[Task(TaskType.FINETUNE, DatasetType.TRAIN)],
)
path = Path(cfg.data_dir)
files = list(path.glob("split_*.jsonl"))

In [12]:
from unlearn_order.pipeline import run_pipeline


run_pipeline(model, tokenizer, cfg)

  4%|▍         | 4/100 [00:36<14:33,  9.09s/it]

Epoch 5 loss: 0.002551023390928271 acc: 1.0
Dataset type: DatasetType.TRAIN, task type: TaskType.FINETUNE, acc: 1.0


,accuracy,dataset_type,task_type
0,1.0,train,finetune
